In [30]:

# importing the requests library
import requests
import pandas as  pd
import pickle
import re

In [31]:
data = pd.read_csv('./gold_raw.csv')

# tweets = data['tweet'].tolist()

In [32]:
data[data['id'] == 1369812663623180288]['tweet']

0    In great news my senior parents are getting th...
Name: tweet, dtype: object

In [33]:
# api-endpoint
ENDPOINT = "https://sentic.net/api/en/"

In [34]:
POLARITY_KEY = 'UBHiejvp0lXkG.py'

In [35]:
URL = ENDPOINT + POLARITY_KEY

In [36]:
# location given here
# text = "I've had my 2nd dose of Covid :Vaccine. I got myself vaccinated to protect my service , my colleagues and myself. As a doctor I am convinced of its safety and efficacy and would encourage everyone else to take up the opportunity to get the vaccine too when it is offered"
text = "In great news my senior parents are getting the vaccine tomorrow. <EMOJI:red_heart> Whew, thank goodness. #vaccine"
# defining a params dict for the parameters to be sent to the API

for c in [';', '&', '#']: 
    text = text.replace(c, ':')
    text = re.sub('<EMOJI[\S]*>', '', text)
    text = re.sub('<USER>', '', text)
    
PARAMS = {'text':text}
  
# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)
print(r.text)

['great news', 'senior', 'vaccine', 'thank', 'goodness', 'vaccine']



In [37]:
import asyncio
import aiohttp

In [24]:
outputs = []
cnt = 0

def preprocess(text):
    for c in [';', '&', '#']: 
        text = text.replace(c, ':')
    text = re.sub('<EMOJI[\S]*>', '', text)
    text = re.sub('<USER>', '', text)
    return text

async def get(session, text, id):
    try:
        async with session.get(url=URL, params = {'text':preprocess(text)}) as response:
            resp = await response.read()
            return [id, resp]
    except Exception as e:
        print("Unable to get url due to {}.".format(e.__class__))


async def main(data):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(session, row['tweet'], row['id']) for idx, row in data.iterrows()])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
    # print(ret)
    outputs.extend(ret)

# l = len(tweets)
loop = asyncio.get_event_loop()
loop.run_until_complete(main(data))
# fl = open('outputs', 'wb')
# pickle.dump(outputs, fl)

Finalized all. Return is a list of len 2000 outputs.


In [38]:
outputs

[[1369812663623180288, 'POSITIVE'],
 [1354514776534220804, 'NEGATIVE'],
 [1349045265945264130, 'POSITIVE'],
 [1366793773728980992, 'POSITIVE'],
 [1365339417997623296, 'NEGATIVE'],
 [1358108212055203844, 'NEGATIVE'],
 [1371004616897269763, 'POSITIVE'],
 [1356666221450690560, 'POSITIVE'],
 [1352375432214142978, 'POSITIVE'],
 [1357212378765684738, 'NEGATIVE'],
 [1356389129781673985, 'NEGATIVE'],
 [1361768521949192198, 'POSITIVE'],
 [1371143426599702528, 'POSITIVE'],
 [1364394016406712324, 'POSITIVE'],
 [1375851778340749313, 'NEGATIVE'],
 [1361389133231517699, 'NEGATIVE'],
 [1347738564688281600, 'POSITIVE'],
 [1353294168039313409, 'NEGATIVE'],
 [1345743906705715201, 'POSITIVE'],
 [1345934230006726656, 'NEGATIVE'],
 [1359788707780448257, 'NEGATIVE'],
 [1360652918718926857, 'POSITIVE'],
 [1368999260767875085, 'POSITIVE'],
 [1362147355458379778, 'POSITIVE'],
 [1355229898240045062, 'POSITIVE'],
 [1369697112728895491, 'NEGATIVE'],
 [1347548987780460545, 'NEGATIVE'],
 [1368909636787703816, 'NEGA

In [26]:
# for i in range(len(outputs)):    
#     outputs[i][1] = outputs[i][1].decode("utf-8")[:-1]
# # map outputs to pandas
# for i in range(len(outputs)):
#     data.loc[data['id'] == outputs[i][0], 'polarity_api_output'] = outputs[i][1]